In [1]:
import random, re, csv
import numpy as np
from sklearn import metrics,svm
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from simdata import *
#from interdata_mysql_simple import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# for kmeans
from scipy import stats
#for evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
#from sklearn.metrics import balanced_accuracy_score


In [2]:
#==============================================
# make mixdata from csv files
#==============================================
mixdata_sim_filedir='../resources/gamedata/mixdata_sim.csv'
interdata_sim_filedir='../resources/gamedata/interdata_sim.csv'
biodata_sim_filedir='../resources/gamedata/biodata_sim.csv'

# mixdata: [[interdata,biodata,prelabel],...,[]]
mixdata=np.asarray(get_mldata(mixdata_sim_filedir),dtype=float)

#mixdata=np.insert(raw_mixdata,[0],mixdata_indx,axis=1)

#interdata_features:4+1 biodata_feature:10+1
interdata=np.asarray(get_mldata(interdata_sim_filedir),dtype=float)
biodata=np.asarray(get_mldata(biodata_sim_filedir),dtype=float)

#==============================================
#prepare features/labels for k-means
#==============================================
mixdata_feature=mixdata[:,0:mixdata[0].shape[0]-1]
mixdata_feature_nd=(mixdata_feature - mixdata_feature.mean(axis=0)) / mixdata_feature.std(axis=0)
mixdata_prelabel=mixdata[:,mixdata[0].shape[0]-1]
#sample_indx=np.arange(raw_mixdata.shape[0]).reshape(raw_mixdata.shape[0],1)
#indx_mixdata_nd=np.insert(mixdata_feature_nd[0:2],[0],sample_indx[0:2],axis=1)

In [25]:
#===================================================================================
# aim: build km cluster and svm classifier from data
# in: bhdata,nbhdata(follow natural distribution);
#     clu_num(number of clusters)
# out: dict={"cluer":cluster_km,"clfer":classifier_svm}
#===================================================================================
def build_analyser(bhdata_nd,nbhdata_nd,clu_num):
    #Cluster from kmeans
    cluster_km = KMeans(n_clusters=clu_num, random_state=0).fit(bhdata_nd)
    #Classifier from SVM
    cluster_result=cluster_km.labels_
    clf_train_feature,\
    clf_test_feature,\
    clf_train_label,\
    clf_test_label = train_test_split(nbhdata_nd,cluster_result, test_size=.2)
    # build classifier
    classifier_svm=svm.SVC(C=1).fit(clf_train_feature,clf_train_label)
    return {"clu":cluster_km,"clf":classifier_svm}

all_anls=build_analyser(mixdata_feature_nd[:,0:4],mixdata_feature_nd[:,5:],3)


In [26]:
clfer=all_anls["clf"]
clker=all_anls['clu']


clf2: 0.72 (+/- 0.11)


In [27]:
level1=clf2.predict(clf_test_feature[0].reshape(1,-1))
pro

In [ ]:




    
num_data=np.arange(0,noisydata.size)
plt.plot(num_data,noisydata,'ro-',label='Noisy Data')
plt.plot(rawdata,'bo-',label='Raw Data')
plt.legend() # 展示图例
plt.xlabel('Offer type') # 给 x 轴添加标签
plt.ylabel('Offer Value') # 给 y 轴添加标签
plt.title('Uniform Noise Analysis') # 添加图形标题
plt.show()

In [ ]:
#=========================
# add noise 
#=========================
# name:get_uninoisylabel
# fun: add uniform noise in to data
# in: rawdata(np.array, NOT normalized), valuerange([low,high+1], range of noise)
# out: noisydata(np.array)
def get_uninoisylabel(rawlabel,noiselevel):
    noisenum=int(noiselevel/100*rawlabel.size)
    uninoise=np.random.randint(0,1,size=noisenum)
    label_uninoise=np.append(uninoise,rawlabel[noisenum:])
    return label_uninoise


In [ ]:
noiselevel=40
#temp_tl=train_labels[0:10]
train_labels_uninoise=get_uninoisylabel(train_labels,noiselevel) 
#============================
#classify interata by SVM
#============================

clf_svm=svm.SVC(C=1)
clf_svm.fit(train_features,train_labels_uninoise)
labels_pre_svm=clf_svm.predict(test_features)
#============================
# Evaluation
#============================

acc_svm=accuracy_score(test_labels,labels_pre_svm)
print("acc_svm:",acc_svm)


In [ ]:
count, bins, ignored = plt.hist(s, 2,normed=True)
plt.plot(bins, np.ones_like(bins), linewidth=2, color='r')
plt.show()
print(s)

In [ ]:
# bhdata=10*3, nbhdata=10*2 
bhdata=np.append(np.arange(15,dtype=float),np.arange(20,35)).reshape(10,3)
nbhdata=np.append(np.arange(10,11,0.1),np.arange(20,23,0.3)).reshape(10,2)
data_indx=np.arange(1,bhdata.shape[0]+1).reshape(bhdata.shape[0],1)

#alldata=[ [indx,bhdata,nhbdata],...,[] ], 10*6
alldata=np.insert(np.insert(bhdata,[3],nbhdata,axis=1),[0],data_indx,axis=1)

print(alldata[:,1:4])

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(bhdata)
kresult= kmeans.predict(bhdata)

In [ ]:
s = np.random.uniform(-1,0,1000)

In [ ]:


behave_features_len=3
behave_features=simdata[:,0:behave_features_len]
behave_features_nd = (behave_features - behave_features.mean(axis=0)) / behave_features.std(axis=0)

train_features_clu,\
test_features_clu,\
train_labels_clu,\
test_labels_clu = train_test_split(behave_features_nd,simdata_l, test_size=.2)

In [ ]:
#========================
# clustering by kmeans
#========================

X = np.array(train_features_clu)
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
kresult= kmeans.predict(test_features_clu)


# 相似度检验
print("similarity test: ",metrics.adjusted_rand_score(kresult,test_labels_clu))


print("train_features_clu: ",train_features_clu)
print("train_labels_clu: ",train_labels_clu)
print("f_inter_nd: ",f_inter_nd)
print("simdata_f: ",simdata_f)
print("simdata_l: ",list(simdata_l))

#=================================
# classification by svm
#=================================

print("simdata: ",np.insert(simdata_f,[5],np.reshape(simdata_l,(5,1)),axis=1))